In [39]:
import pandas as pd
import mysql.connector as sql
import streamlit as st
import plotly.express as px
import os
import json
from PIL import Image
import git
from git.repo.base import Repo
import subprocess

In [40]:
#clone data


repo_url = "https://github.com/PhonePe/pulse.git"
destination_path = "D:/Projects/PhonePe"  # Use a valid path without special characters

subprocess.run(["git", "clone", repo_url, destination_path])

CompletedProcess(args=['git', 'clone', 'https://github.com/PhonePe/pulse.git', 'D:/Projects/PhonePe'], returncode=128)

# for aggregated transaction :

In [41]:
# Define the path to the directory containing the aggregated transaction data:
path1 = "D:/Projects/PhonePe/data/aggregated/transaction/country/india/state/"

#Get the list of directories inside the specified path:
agg_trans_list = os.listdir(path1)# The os.listdir() function returns a list of all the items (directories and files) present in the specified directory.


#Create an empty dictionary to store the data:
columns1 = {'State':[] , 'Year':[] , 'Quarter':[] , 'Transaction_type':[] , 'Transaction_count':[] , 'Transaction_amount':[]} #This dictionary, named columns1, will be used to collect the extracted data.

#Iterate over the directories in agg_trans_list:
for state in agg_trans_list:
    cur_state = path1 + state + "/"        # This loop iterates over each directory in agg_trans_list and concatenates the current state directory's path to the cur_state variable.
    
    if not os.path.isdir(cur_state):
        continue
    
    agg_year_list = os.listdir(cur_state)
    
    for year in agg_year_list:
        cur_year = cur_state + year + "/"
       
        if not os.path.isdir(cur_year):
            continue
       
        agg_file_list = os.listdir(cur_year)
        
        for file in agg_file_list:
            cur_file = cur_year + file 
            
            if not os.path.isfile(cur_file):
                continue
                
                
            #Open and read the JSON file:
            data = open(cur_file,'r')
            A = json.load(data)
            
            #Extract the relevant information from the JSON data and append it to the columns1 dictionary:
            for i in A['data']['transactionData']:
                name = i['name']
                count = i['paymentInstruments'][0]['count']
                amount = i['paymentInstruments'][0]['amount']
                columns1['Transaction_type'].append(name)
                columns1['Transaction_count'].append(count)
                columns1['Transaction_amount'].append(amount)
                columns1['State'].append(state)
                columns1['Year'].append(year)
                columns1['Quarter'].append(int(file.strip('.json')))
                
df_agg_trans = pd.DataFrame(columns1)              
df_agg_trans.shape

(3954, 6)

In [42]:
df = df_agg_trans
df

,State,Year,Quarter,Transaction_type,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,Recharge & bill payments,4200,1.845307e+06
1,andaman-&-nicobar-islands,2018,1,Peer-to-peer payments,1871,1.213866e+07
2,andaman-&-nicobar-islands,2018,1,Merchant payments,298,4.525072e+05
3,andaman-&-nicobar-islands,2018,1,Financial Services,33,1.060142e+04
4,andaman-&-nicobar-islands,2018,1,Others,256,1.846899e+05
...,...,...,...,...,...,...
3949,west-bengal,2023,2,Merchant payments,245111000,1.767046e+11
3950,west-bengal,2023,2,Peer-to-peer payments,240347041,7.970548e+11
3951,west-bengal,2023,2,Recharge & bill payments,58950434,3.478924e+10
3952,west-bengal,2023,2,Financial Services,327537,3.174670e+08


In [43]:
print(df_agg_trans.columns)

Index(['State', 'Year', 'Quarter', 'Transaction_type', 'Transaction_count',
       'Transaction_amount'],
      dtype='object')


In [44]:
# Count the number of null values in each column
print(df.isnull().sum())

State                 0
Year                  0
Quarter               0
Transaction_type      0
Transaction_count     0
Transaction_amount    0
dtype: int64


# For aggregated user :

In [45]:
# Define the path to the directory containing the aggregated transaction data:
path2 = "D:/Projects/PhonePe/data/aggregated/user/country/india/state/"

agg_user_list = os.listdir(path2)

columns2 = {'State': [], 'Year': [], 'Quarter': [], 'Brands': [], 'Count': [],
            'Percentage': []}
for state in agg_user_list:
    cur_state = path2 + state + "/"
    agg_year_list = os.listdir(cur_state)
    
    for year in agg_year_list:
        cur_year = cur_state + year + "/"
        agg_file_list = os.listdir(cur_year)

        for file in agg_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            B = json.load(data)
            try:
                for i in B["data"]["usersByDevice"]:
                    brand_name = i["brand"]
                    counts = i["count"]
                    percents = i["percentage"]
                    columns2["Brands"].append(brand_name)
                    columns2["Count"].append(counts)
                    columns2["Percentage"].append(percents)
                    columns2["State"].append(state)
                    columns2["Year"].append(year)
                    columns2["Quarter"].append(int(file.strip('.json')))
            except:
                pass
df_agg_user = pd.DataFrame(columns2)

In [46]:
df_agg_user

,State,Year,Quarter,Brands,Count,Percentage
0,andaman-&-nicobar-islands,2018,1,Xiaomi,1665,0.247033
1,andaman-&-nicobar-islands,2018,1,Samsung,1445,0.214392
2,andaman-&-nicobar-islands,2018,1,Vivo,982,0.145697
3,andaman-&-nicobar-islands,2018,1,Oppo,501,0.074332
4,andaman-&-nicobar-islands,2018,1,OnePlus,332,0.049258
...,...,...,...,...,...,...
6727,west-bengal,2022,1,Lenovo,330017,0.015056
6728,west-bengal,2022,1,Infinix,284678,0.012987
6729,west-bengal,2022,1,Asus,280347,0.012790
6730,west-bengal,2022,1,Apple,277752,0.012671


In [47]:
df_agg_user.columns

Index(['State', 'Year', 'Quarter', 'Brands', 'Count', 'Percentage'], dtype='object')

In [48]:
df1=df_agg_user

In [49]:
# Count the number of null values in each column
print(df1.isnull().sum())

State         0
Year          0
Quarter       0
Brands        0
Count         0
Percentage    0
dtype: int64


# For map transactions

In [50]:
path3 = "D:/Projects/PhonePe/data/map/transaction/hover/country/india/state/"

map_trans_list = os.listdir(path3)

columns3 = {'State': [], 'Year': [], 'Quarter': [], 'District': [], 'Count': [],
            'Amount': []}

for state in map_trans_list:
    cur_state = path3 + state + "/"
    map_year_list = os.listdir(cur_state)
    
    for year in map_year_list:
        cur_year = cur_state + year + "/"
        map_file_list = os.listdir(cur_year)
        
        for file in map_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            C = json.load(data)
            
            for i in C["data"]["hoverDataList"]:
                district = i["name"]
                count = i["metric"][0]["count"]
                amount = i["metric"][0]["amount"]
                columns3["District"].append(district)
                columns3["Count"].append(count)
                columns3["Amount"].append(amount)
                columns3['State'].append(state)
                columns3['Year'].append(year)
                columns3['Quarter'].append(int(file.strip('.json')))
                
df_map_trans = pd.DataFrame(columns3)

df_map_trans

,State,Year,Quarter,District,Count,Amount
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,442,9.316631e+05
1,andaman-&-nicobar-islands,2018,1,south andaman district,5688,1.256025e+07
2,andaman-&-nicobar-islands,2018,1,nicobars district,528,1.139849e+06
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,825,1.317863e+06
4,andaman-&-nicobar-islands,2018,2,south andaman district,9395,2.394824e+07
...,...,...,...,...,...,...
16095,west-bengal,2023,2,nadia district,26194658,5.410386e+10
16096,west-bengal,2023,2,birbhum district,14912523,2.941312e+10
16097,west-bengal,2023,2,purba medinipur district,30724502,6.595899e+10
16098,west-bengal,2023,2,maldah district,24677357,5.218621e+10


In [51]:
df3 = df_map_trans

# Count the number of null values in each column
print(df3.isnull().sum())

State       0
Year        0
Quarter     0
District    0
Count       0
Amount      0
dtype: int64


In [52]:
print(df_map_trans.columns)

Index(['State', 'Year', 'Quarter', 'District', 'Count', 'Amount'], dtype='object')


# For Dataframe of map user

In [53]:
# For Dataframe of map user

path4 = "D:/Projects/PhonePe/data/map/user/hover/country/india/state/"

map_user_list = os.listdir(path4)

columns4 = {"State": [], "Year": [], "Quarter": [], "District": [],
            "RegisteredUser": [], "AppOpens": []}

for state in map_user_list:
    cur_state = path4 + state + "/"
    map_year_list = os.listdir(cur_state)
    
    for year in map_year_list:
        cur_year = cur_state + year + "/"
        map_file_list = os.listdir(cur_year)
        
        for file in map_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            D = json.load(data)
            
            for i in D["data"]["hoverData"].items():
                district = i[0]
                registereduser = i[1]["registeredUsers"]
                appOpens = i[1]['appOpens']
                columns4["District"].append(district)
                columns4["RegisteredUser"].append(registereduser)
                columns4["AppOpens"].append(appOpens)
                columns4['State'].append(state)
                columns4['Year'].append(year)
                columns4['Quarter'].append(int(file.strip('.json')))
                
df_map_user = pd.DataFrame(columns4)

df_map_user

,State,Year,Quarter,District,RegisteredUser,AppOpens
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,632,0
1,andaman-&-nicobar-islands,2018,1,south andaman district,5846,0
2,andaman-&-nicobar-islands,2018,1,nicobars district,262,0
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,911,0
4,andaman-&-nicobar-islands,2018,2,south andaman district,8143,0
...,...,...,...,...,...,...
16099,west-bengal,2023,2,nadia district,1482400,9509833
16100,west-bengal,2023,2,birbhum district,934943,12789154
16101,west-bengal,2023,2,purba medinipur district,1468608,12096564
16102,west-bengal,2023,2,maldah district,1046132,17662446


In [54]:
print(df_map_user.columns)

Index(['State', 'Year', 'Quarter', 'District', 'RegisteredUser', 'AppOpens'], dtype='object')


# Dataframe of top transactions

In [55]:
path5 = "D:/Projects/PhonePe/data/top/transaction/country/india/state/"

top_trans_list = os.listdir(path5)
columns5 = {'State': [], 'Year': [], 'Quarter': [], 'Pincode': [], 'Transaction_count': [],
            'Transaction_amount': []}

for state in top_trans_list:
    cur_state = path5 + state + "/"
    top_year_list = os.listdir(cur_state)
    
    for year in top_year_list:
        cur_year = cur_state + year + "/"
        top_file_list = os.listdir(cur_year)
        
        for file in top_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            E = json.load(data)
            
            for i in E['data']['pincodes']:
                name = i['entityName']
                count = i['metric']['count']
                amount = i['metric']['amount']
                columns5['Pincode'].append(name)
                columns5['Transaction_count'].append(count)
                columns5['Transaction_amount'].append(amount)
                columns5['State'].append(state)
                columns5['Year'].append(year)
                columns5['Quarter'].append(int(file.strip('.json')))
df_top_trans = pd.DataFrame(columns5)
            
df_top_trans

,State,Year,Quarter,Pincode,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,744101,1622,2.769298e+06
1,andaman-&-nicobar-islands,2018,1,744103,1223,2.238042e+06
2,andaman-&-nicobar-islands,2018,1,744102,969,3.519060e+06
3,andaman-&-nicobar-islands,2018,1,744105,685,1.298561e+06
4,andaman-&-nicobar-islands,2018,1,744104,340,1.039715e+06
...,...,...,...,...,...,...
7848,west-bengal,2023,2,700001,2918444,6.259877e+09
7849,west-bengal,2023,2,721301,2910870,4.458107e+09
7850,west-bengal,2023,2,700039,2886562,4.662887e+09
7851,west-bengal,2023,2,700015,2774383,4.372975e+09


In [56]:
print(df_top_trans.columns)

Index(['State', 'Year', 'Quarter', 'Pincode', 'Transaction_count',
       'Transaction_amount'],
      dtype='object')


In [57]:
# For Dataframe of top users

path6 = "D:/Projects/PhonePe/data/top/user/country/india/state/"

top_user_list = os.listdir(path6)
columns6 = {'State': [], 'Year': [], 'Quarter': [], 'Pincode': [],
            'RegisteredUsers': []}

for state in top_user_list:
    cur_state = path6 + state + "/"
    top_year_list = os.listdir(cur_state)
    
    for year in top_year_list:
        cur_year = cur_state + year + "/"
        top_file_list = os.listdir(cur_year)
        
        for file in top_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            F = json.load(data)
            
            for i in F['data']['pincodes']:
                name = i['name']
                registeredUsers = i['registeredUsers']
                columns6['Pincode'].append(name)
                columns6['RegisteredUsers'].append(registeredUsers)
                columns6['State'].append(state)
                columns6['Year'].append(year)
                columns6['Quarter'].append(int(file.strip('.json')))
df_top_user = pd.DataFrame(columns6)         
df_top_user

,State,Year,Quarter,Pincode,RegisteredUsers
0,andaman-&-nicobar-islands,2018,1,744103,1608
1,andaman-&-nicobar-islands,2018,1,744101,1108
2,andaman-&-nicobar-islands,2018,1,744105,1075
3,andaman-&-nicobar-islands,2018,1,744102,1006
4,andaman-&-nicobar-islands,2018,1,744104,272
...,...,...,...,...,...
7849,west-bengal,2023,2,700015,117690
7850,west-bengal,2023,2,742304,114746
7851,west-bengal,2023,2,721101,114293
7852,west-bengal,2023,2,700150,110694


In [58]:
print(df_top_user.columns)

Index(['State', 'Year', 'Quarter', 'Pincode', 'RegisteredUsers'], dtype='object')


In [59]:
df_agg_trans.to_csv('agg_trans.csv',index=False)
df_agg_user.to_csv('agg_user.csv',index=False)
df_map_trans.to_csv('map_trans.csv',index=False)
df_map_user.to_csv('map_user.csv',index=False)
df_top_trans.to_csv('top_trans.csv',index=False)
df_top_user.to_csv('top_user.csv',index=False)

    
df_agg_trans.to_csv('D:/Project/2PHONEPE/phonepe extarct from sql/agg_trans.csv',index=False)
df_agg_user.to_csv('D:/Project/2PHONEPE/phonepe extarct from sql/agg_user.csv',index=False)
df_map_trans.to_csv('D:/Project/2PHONEPE/phonepe extarct from sql/map_trans.csv',index=False)
df_map_user.to_csv('D:/Project/2PHONEPE/phonepe extarct from sql/map_user.csv',index=False)
df_top_trans.to_csv('D:/Project/2PHONEPE/phonepe extarct from sql/top_trans.csv',index=False)
df_top_user.to_csv('D:/Project/2PHONEPE/phonepe extarct from sql/top_user.csv',index=False)

# Connecting with SQL

In [60]:
try:
    mydb = sql.connect(
        host="localhost",
        user="root",
        password="Dhinesh!1",
        database="phonepe_pulse"
    )
    
    print("Connection established successfully!")
    
except sql.Error as error:
    print("Error while connecting to MySQL:", error)

# Create a cursor object    
mycursor = mydb.cursor(buffered=True) 

Connection established successfully!


In [61]:
# 1.Create the agg_trans table
create_table_query = """CREATE TABLE IF NOT EXISTS agg_trans (
    State VARCHAR(100),
    Year INT,
    Quarter INT,
    Transaction_type VARCHAR(100),
    Transaction_count INT,
    Transaction_amount DOUBLE
)
"""
mycursor.execute(create_table_query)
print("agg_trans table created successfully!")

agg_trans table created successfully!


In [62]:
# Insert rows from DataFrame into the agg_trans table
insert_query = """
INSERT INTO agg_trans (State, Year, Quarter, Transaction_type, Transaction_count, Transaction_amount)
VALUES (%s, %s, %s, %s, %s, %s)
"""

for _, row in df_agg_trans.iterrows():
    data = (row['State'], row['Year'], row['Quarter'], row['Transaction_type'], row['Transaction_count'], row['Transaction_amount'])
    mycursor.execute(insert_query, data)
    
#The code uses a loop to iterate over the rows of the DataFrame df_agg_trans. The loop variable _ is used to ignore the index of the DataFrame, while row represents each row of the DataFrame.

# Commit the changes and close the connection
mydb.commit()

In [63]:
# 2 .Create the agg_user table
create_table_query = """
CREATE TABLE IF NOT EXISTS agg_user (
    State VARCHAR(100),
    Year INT,
    Quarter INT,
    Brands varchar(100),
    Count INT,
    Percentage DOUBLE
)
"""
mycursor.execute(create_table_query)
print("agg_user table created successfully!")

agg_user table created successfully!


In [64]:
# Insert rows from DataFrame into the agg_trans table
insert_query = """
INSERT INTO agg_user (State, Year, Quarter, Brands, Count, Percentage)
VALUES (%s, %s, %s, %s, %s, %s)
"""

for _, row in df_agg_user.iterrows():
    data = (row['State'], row['Year'], row['Quarter'], row['Brands'], row['Count'], row['Percentage'])
    mycursor.execute(insert_query, data)
    
#The code uses a loop to iterate over the rows of the DataFrame df_agg_trans. The loop variable _ is used to ignore the index of the DataFrame, while row represents each row of the DataFrame.

# Commit the changes and close the connection
mydb.commit()

In [65]:
# 3 .Create the map_trans table
create_table_query = """
CREATE TABLE IF NOT EXISTS map_trans (
    State VARCHAR(100),
    Year INT,
    Quarter INT,
    District varchar(100),
    Count INT,
    Amount DOUBLE
)
"""
mycursor.execute(create_table_query)
print("map_trans table created successfully!")

map_trans table created successfully!


In [66]:
# Insert rows from DataFrame into the map_trans table
insert_query = """
INSERT INTO map_trans (State, Year, Quarter, District, Count, Amount)
VALUES (%s, %s, %s, %s, %s, %s)
"""

for _, row in df_map_trans.iterrows():
    data = (row['State'], row['Year'], row['Quarter'], row['District'], row['Count'], row['Amount'])
    mycursor.execute(insert_query, data)
    
#The code uses a loop to iterate over the rows of the DataFrame df_agg_trans. The loop variable _ is used to ignore the index of the DataFrame, while row represents each row of the DataFrame.

# Commit the changes and close the connection
mydb.commit()

In [67]:
# 4 .Create the map_user table
create_table_query = """
CREATE TABLE IF NOT EXISTS map_user (
    State VARCHAR(100),
    Year INT,
    Quarter INT,
    District varchar(100),
    RegisteredUser INT,
    AppOpens INT
)
"""
mycursor.execute(create_table_query)
print("map_user table created successfully!")

map_user table created successfully!


In [68]:
print(df_map_user.columns)

Index(['State', 'Year', 'Quarter', 'District', 'RegisteredUser', 'AppOpens'], dtype='object')


In [69]:
# Insert rows from DataFrame into the map_user table
insert_query = """
INSERT INTO map_user (State, Year, Quarter, District, RegisteredUser, AppOpens)
VALUES (%s, %s, %s, %s, %s, %s)
"""

for _, row in df_map_user.iterrows():
    data = (row['State'], row['Year'], row['Quarter'], row['District'], row['RegisteredUser'], row['AppOpens'])
    mycursor.execute(insert_query, data)
    
#The code uses a loop to iterate over the rows of the DataFrame df_agg_trans. The loop variable _ is used to ignore the index of the DataFrame, while row represents each row of the DataFrame.

# Commit the changes and close the connection
mydb.commit()

In [70]:
# 5 .Create the top_trans table
create_table_query = """
CREATE TABLE IF NOT EXISTS top_trans (
    State VARCHAR(100),
    Year INT,
    Quarter INT,
    Pincode INT,
    Transaction_count INT,
    Transaction_amount DOUBLE
)
"""
mycursor.execute(create_table_query)
print("top_trans table created successfully!")

top_trans table created successfully!


In [71]:
# Insert rows from DataFrame into the top_trans table
insert_query = """
INSERT INTO top_trans (State, Year, Quarter, Pincode, Transaction_count, Transaction_amount)
VALUES (%s, %s, %s, %s, %s, %s)
"""

for _, row in df_top_trans.iterrows():
    data = (row['State'], row['Year'], row['Quarter'], row['Pincode'], row['Transaction_count'], row['Transaction_amount'])
    mycursor.execute(insert_query, data)
    
#The code uses a loop to iterate over the rows of the DataFrame df_agg_trans. The loop variable _ is used to ignore the index of the DataFrame, while row represents each row of the DataFrame.

# Commit the changes and close the connection
mydb.commit()

In [72]:
print(df_top_user.columns)

Index(['State', 'Year', 'Quarter', 'Pincode', 'RegisteredUsers'], dtype='object')


In [73]:
# 6 .Create the top_user table
create_table_query = """
CREATE TABLE IF NOT EXISTS top_user (
    State VARCHAR(100),
    Year INT,
    Quarter INT,
    Pincode INT,
    RegisteredUsers INT
    )
"""
mycursor.execute(create_table_query)
print("top_user table created successfully!")

top_user table created successfully!


In [74]:
# Insert rows from DataFrame into the top_user table
insert_query = """
INSERT INTO top_user (State, Year, Quarter, Pincode, RegisteredUsers)
VALUES (%s, %s, %s, %s, %s)
"""

for _, row in df_top_user.iterrows():
    data = (row['State'], row['Year'], row['Quarter'], row['Pincode'], row['RegisteredUsers'])
    mycursor.execute(insert_query, data)
    
#The code uses a loop to iterate over the rows of the DataFrame df_agg_trans. The loop variable _ is used to ignore the index of the DataFrame, while row represents each row of the DataFrame.

# Commit the changes and close the connection
mydb.commit()

In [75]:
# show databases tables
mycursor.execute("show tables")
mycursor.fetchall()

[('agg_trans',),
 ('agg_user',),
 ('map_trans',),
 ('map_user',),
 ('top_trans',),
 ('top_user',)]

In [76]:
# Close the cursor and database connection
mycursor.close()
mydb.close()